In [1]:
"""Baseline model. 1 layer LSTM with a classification layer trained for 100 epochs"""

'Baseline model. 1 layer LSTM with a classification layer trained for 100 epochs'

In [2]:
import sys
import os
import math

import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("../../data/processed/signalEUR_USD_Labelled_v1_processed.csv", index_col=0)

In [4]:
df.head(5)

,Date,Price,Open,High,Low,Change %,diff_1,label,sma,ema,cma,macd,macd_s,macd_h,roc,rsi,Bollinger_up,Bollinger_down,cci
0,2013-01-01,1.3205,1.3194,1.3220,1.3180,0.07%,NaN,NaN,NaN,1.320500,1.320500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-01-02,1.3186,1.3205,1.3302,1.3157,-0.14%,-0.0019,0.0,NaN,1.318956,1.319550,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
2,2013-01-03,1.3048,1.3187,1.3192,1.3046,-1.05%,-0.0138,1.0,NaN,1.307931,1.314633,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
3,2013-01-04,1.3069,1.3048,1.3091,1.2999,0.16%,0.0021,2.0,NaN,1.307136,1.312700,NaN,NaN,NaN,NaN,13.559663,NaN,NaN,NaN
4,2013-01-07,1.3117,1.3072,1.3121,1.3018,0.37%,0.0048,2.0,NaN,1.310649,1.312500,NaN,NaN,NaN,NaN,36.329386,NaN,NaN,NaN


In [5]:
import keras

In [40]:
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

def prepare_data(df):
    df["label_3days"] = df["label"].shift(-3)
    df["label_5days"] = df["label"].shift(-5)
    df = df.dropna(axis=0)
    df = df.reset_index(drop=True)

    time = ["Date"]
    var = ["Price",  "sma", "ema", "cma", "macd",	"macd_s", 
               "macd_h", "roc", "rsi",	"Bollinger_up", "Bollinger_down", "cci"]
    columns = []
    for v in var:
        columns.append(v)
        columns.append(v + "(t-2)")
        columns.append(v + "(t-1)")
    
    labels = ["label", "label_3days", "label_5days"]
    features = df[time + columns + labels]

    X = features.drop(labels= time + labels, axis=1)
    print(X.head(5))
    Y = features[["label"]]

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    onehot_encoder = OneHotEncoder()
    Y = to_categorical(Y)
    
    return X, Y, features
    

In [52]:
from pandas import concat

def window(df):
    """Given time series data, window it such that we can get (t-2), (t-1) and (t)"""
    n_var = df.shape[1]
    cols, names = list(), list()

    n_prev = 2
    for i in range(n_prev, -1, -1):
        cols.append(df.shift(i))
        if i > 0:
            names += [(f"{j}(t-{i})") for j in list(df.columns)]
        else:
            names += [(f"{j}") for j in list(df.columns)]
    agg = concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg
agg = window(df)
pd.set_option('display.max_columns', None)
agg



,Date(t-2),Price(t-2),Open(t-2),High(t-2),Low(t-2),Change %(t-2),diff_1(t-2),label(t-2),sma(t-2),ema(t-2),cma(t-2),macd(t-2),macd_s(t-2),macd_h(t-2),roc(t-2),rsi(t-2),Bollinger_up(t-2),Bollinger_down(t-2),cci(t-2),label_3days(t-2),label_5days(t-2),Date(t-1),Price(t-1),Open(t-1),High(t-1),Low(t-1),Change %(t-1),diff_1(t-1),label(t-1),sma(t-1),ema(t-1),cma(t-1),macd(t-1),macd_s(t-1),macd_h(t-1),roc(t-1),rsi(t-1),Bollinger_up(t-1),Bollinger_down(t-1),cci(t-1),label_3days(t-1),label_5days(t-1),Date,Price,Open,High,Low,Change %,diff_1,label,sma,ema,cma,macd,macd_s,macd_h,roc,rsi,Bollinger_up,Bollinger_down,cci,label_3days,label_5days
35,2013-02-15,1.3363,1.3363,1.3394,1.3306,0.01%,0.0002,0.0,1.344735,1.336730,1.335315,0.002701,0.005461,-0.002760,-0.000150,39.939646,1.364281,1.325189,-62.849696,1.0,0.0,2013-02-18,1.3352,1.3357,1.3377,1.3321,-0.08%,-0.0011,0.0,1.344925,1.335553,1.335311,0.001906,0.004750,-0.002844,-0.004102,38.588551,1.363994,1.325856,-68.561252,1.0,1.0,2013-02-19,1.3388,1.3352,1.3397,1.3329,0.27%,0.0036,2.0,1.345260,1.338051,1.335408,0.001549,0.004109,-0.002561,-0.004906,45.544851,1.363601,1.326919,-57.039491,0.0,0.0
36,2013-02-18,1.3352,1.3357,1.3377,1.3321,-0.08%,-0.0011,0.0,1.344925,1.335553,1.335311,0.001906,0.004750,-0.002844,-0.004102,38.588551,1.363994,1.325856,-68.561252,1.0,1.0,2013-02-19,1.3388,1.3352,1.3397,1.3329,0.27%,0.0036,2.0,1.345260,1.338051,1.335408,0.001549,0.004109,-0.002561,-0.004906,45.544851,1.363601,1.326919,-57.039491,0.0,0.0,2013-02-20,1.3281,1.3387,1.3434,1.3270,-0.80%,-0.0107,1.0,1.345075,1.330396,1.335211,0.000398,0.003367,-0.002969,-0.012785,32.802173,1.364052,1.326098,-90.131403,1.0,2.0
37,2013-02-19,1.3388,1.3352,1.3397,1.3329,0.27%,0.0036,2.0,1.345260,1.338051,1.335408,0.001549,0.004109,-0.002561,-0.004906,45.544851,1.363601,1.326919,-57.039491,0.0,0.0,2013-02-20,1.3281,1.3387,1.3434,1.3270,-0.80%,-0.0107,1.0,1.345075,1.330396,1.335211,0.000398,0.003367,-0.002969,-0.012785,32.802173,1.364052,1.326098,-90.131403,1.0,2.0,2013-02-21,1.3189,1.3283,1.3291,1.3161,-0.69%,-0.0092,1.0,1.344140,1.321553,1.334782,-0.001242,0.002445,-0.003688,-0.012873,25.675429,1.366251,1.322029,-149.263447,0.0,1.0
38,2013-02-20,1.3281,1.3387,1.3434,1.3270,-0.80%,-0.0107,1.0,1.345075,1.330396,1.335211,0.000398,0.003367,-0.002969,-0.012785,32.802173,1.364052,1.326098,-90.131403,1.0,2.0,2013-02-21,1.3189,1.3283,1.3291,1.3161,-0.69%,-0.0092,1.0,1.344140,1.321553,1.334782,-0.001242,0.002445,-0.003688,-0.012873,25.675429,1.366251,1.322029,-149.263447,0.0,1.0,2013-02-22,1.3190,1.3190,1.3247,1.3146,0.01%,0.0001,0.0,1.342780,1.319589,1.334377,-0.002505,0.001455,-0.003960,-0.012946,25.877405,1.367545,1.318015,-135.430243,2.0,1.0
39,2013-02-21,1.3189,1.3283,1.3291,1.3161,-0.69%,-0.0092,1.0,1.344140,1.321553,1.334782,-0.001242,0.002445,-0.003688,-0.012873,25.675429,1.366251,1.322029,-149.263447,0.0,1.0,2013-02-22,1.3190,1.3190,1.3247,1.3146,0.01%,0.0001,0.0,1.342780,1.319589,1.334377,-0.002505,0.001455,-0.003960,-0.012946,25.877405,1.367545,1.318015,-135.430243,2.0,1.0,2013-02-25,1.3062,1.3175,1.3319,1.3047,-0.97%,-0.0128,1.0,1.340810,1.309290,1.333673,-0.004487,0.000267,-0.004754,-0.021720,18.466037,1.370424,1.311196,-136.799244,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2339,2021-12-16,1.1328,1.1286,1.1361,1.1280,0.38%,0.0043,2.0,1.128465,1.131698,1.181217,-0.003936,-0.005225,0.001289,0.003188,55.445142,1.136545,1.120385,74.514054,0.0,0.0,2021-12-17,1.1239,1.1330,1.1350,1.1235,-0.79%,-0.0089,1.0,1.128215,1.125699,1.181192,-0.004130,-0.005006,0.000876,-0.006365,39.869634,1.136544,1.119886,-18.249815,2.0,0.0,2021-12-20,1.1275,1.1242,1.1305,1.1234,0.32%,0.0036,2.0,1.128420,1.127084,1.181169,-0.003948,-0.004795,0.000847,-0.000709,46.839553,1.136446,1.120394,-27.330869,0.0,0.0
2340,2021-12-17,1.1239,1.1330,

In [53]:
from sklearn.model_selection import train_test_split

X, Y, features = prepare_data(window(df))

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=420)
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)

    Price  Price(t-2)  Price(t-1)       sma  sma(t-2)  sma(t-1)       ema  \
0  1.3388      1.3363      1.3352  1.345260  1.344735  1.344925  1.338051   
1  1.3281      1.3352      1.3388  1.345075  1.344925  1.345260  1.330396   
2  1.3189      1.3388      1.3281  1.344140  1.345260  1.345075  1.321553   
3  1.3190      1.3281      1.3189  1.342780  1.345075  1.344140  1.319589   
4  1.3062      1.3189      1.3190  1.340810  1.344140  1.342780  1.309290   

   ema(t-2)  ema(t-1)       cma  cma(t-2)  cma(t-1)      macd  macd(t-2)  \
0  1.336730  1.335553  1.335408  1.335315  1.335311  0.001549   0.002701   
1  1.335553  1.338051  1.335211  1.335311  1.335408  0.000398   0.001906   
2  1.338051  1.330396  1.334782  1.335408  1.335211 -0.001242   0.001549   
3  1.330396  1.321553  1.334377  1.335211  1.334782 -0.002505   0.000398   
4  1.321553  1.319589  1.333673  1.334782  1.334377 -0.004487  -0.001242   

   macd(t-1)    macd_s  macd_s(t-2)  macd_s(t-1)    macd_h  macd_h(t-2)  \
0   0

In [44]:
from keras.models import Sequential
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.losses import CategoricalCrossentropy

h_dim = 100
num_classes = 3

model = Sequential()
model.add(LSTM(units=h_dim, input_shape=(X_train.shape[1], 1)))
model.add(Dense(num_classes))
model.compile(loss=CategoricalCrossentropy(from_logits=True), optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 100)               40800     
                                                                 
 dense_2 (Dense)             (None, 3)                 303       
                                                                 
Total params: 41,103
Trainable params: 41,103
Non-trainable params: 0
_________________________________________________________________


In [45]:
epochs = 100

model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, batch_size=64)

Epoch 1/100
29/29 [==============================] - 2s 28ms/step - loss: 1.1008 - accuracy: 0.3370 - val_loss: 1.0972 - val_accuracy: 0.3297
Epoch 2/100
29/29 [==============================] - 1s 19ms/step - loss: 1.0957 - accuracy: 0.3440 - val_loss: 1.0936 - val_accuracy: 0.3536
Epoch 3/100
29/29 [==============================] - 1s 20ms/step - loss: 1.0940 - accuracy: 0.3630 - val_loss: 1.0917 - val_accuracy: 0.3666
Epoch 4/100
29/29 [==============================] - 1s 20ms/step - loss: 1.0896 - accuracy: 0.3543 - val_loss: 1.0916 - val_accuracy: 0.3774
Epoch 5/100
29/29 [==============================] - 1s 19ms/step - loss: 1.0835 - accuracy: 0.3852 - val_loss: 1.1287 - val_accuracy: 0.3492
Epoch 6/100
29/29 [==============================] - 1s 19ms/step - loss: 1.0832 - accuracy: 0.4010 - val_loss: 1.0901 - val_accuracy: 0.3514
Epoch 7/100
29/29 [==============================] - 1s 20ms/step - loss: 1.0795 - accuracy: 0.4010 - val_loss: 1.0840 - val_accuracy: 0.3796
Epoch 

29/29 [==============================] - 1s 20ms/step - loss: 0.2854 - accuracy: 0.8844 - val_loss: 0.3349 - val_accuracy: 0.8482
Epoch 59/100
29/29 [==============================] - 1s 21ms/step - loss: 0.2375 - accuracy: 0.9012 - val_loss: 0.3578 - val_accuracy: 0.8460
Epoch 60/100
29/29 [==============================] - 1s 21ms/step - loss: 0.2254 - accuracy: 0.9148 - val_loss: 0.3343 - val_accuracy: 0.8568
Epoch 61/100
29/29 [==============================] - 1s 22ms/step - loss: 0.2198 - accuracy: 0.9078 - val_loss: 0.3308 - val_accuracy: 0.8612
Epoch 62/100
29/29 [==============================] - 1s 21ms/step - loss: 0.2025 - accuracy: 0.9246 - val_loss: 0.3325 - val_accuracy: 0.8503
Epoch 63/100
29/29 [==============================] - 1s 22ms/step - loss: 0.2675 - accuracy: 0.8855 - val_loss: 0.3653 - val_accuracy: 0.8330
Epoch 64/100
29/29 [==============================] - 1s 20ms/step - loss: 0.2179 - accuracy: 0.9029 - val_loss: 0.3776 - val_accuracy: 0.8460
Epoch 65/100

In [7]:
# Without price as a factor

In [48]:
X = features.drop(labels= ["Date"] + ["label", "label_3days", "label_5days"] + ["Price"], axis=1)

scaler = StandardScaler()
X = scaler.fit_transform(X)
X

array([[ 1.75711290e+00,  1.74688028e+00,  1.85073459e+00, ...,
        -5.82267337e-01, -6.48549048e-01, -7.15253591e-01],
       [ 1.74482298e+00,  1.78712648e+00,  1.84866607e+00, ...,
        -9.65166121e-01, -7.14635687e-01, -5.81952273e-01],
       [ 1.78504452e+00,  1.66750584e+00,  1.83821161e+00, ...,
        -1.64936920e+00, -5.81320962e-01, -9.64810006e-01],
       ...,
       [-5.66798082e-01, -5.95224900e-01, -5.71009894e-01, ...,
        -3.10471530e-01, -1.79884284e-03, -1.28458797e-01],
       [-5.95846969e-01, -5.63922301e-01, -5.73302048e-01, ...,
         9.39908017e-01, -1.27781876e-01, -3.10185606e-01],
       [-5.64563552e-01, -5.15850452e-01, -5.76097357e-01, ...,
        -1.33441060e-01, -3.09526962e-01,  9.40059888e-01]])

In [49]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=420)
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)

(1843, 35) (461, 35)
(1843, 3) (461, 3)


In [50]:
model = Sequential()
model.add(LSTM(units=h_dim, input_shape=(X_train.shape[1], 1)))
model.add(Dense(num_classes))
model.compile(loss=CategoricalCrossentropy(from_logits=True), optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 100)               40800     
                                                                 
 dense_3 (Dense)             (None, 3)                 303       
                                                                 
Total params: 41,103
Trainable params: 41,103
Non-trainable params: 0
_________________________________________________________________


In [51]:
epochs = 100

model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, batch_size=64)

Epoch 1/100
29/29 [==============================] - 2s 27ms/step - loss: 1.0996 - accuracy: 0.3435 - val_loss: 1.0978 - val_accuracy: 0.3341
Epoch 2/100
29/29 [==============================] - 1s 19ms/step - loss: 1.0965 - accuracy: 0.3397 - val_loss: 1.0961 - val_accuracy: 0.3275
Epoch 3/100
29/29 [==============================] - 1s 19ms/step - loss: 1.0943 - accuracy: 0.3576 - val_loss: 1.0927 - val_accuracy: 0.3449
Epoch 4/100
29/29 [==============================] - 1s 19ms/step - loss: 1.0906 - accuracy: 0.3570 - val_loss: 1.0894 - val_accuracy: 0.3818
Epoch 5/100
29/29 [==============================] - 1s 19ms/step - loss: 1.0869 - accuracy: 0.3874 - val_loss: 1.0890 - val_accuracy: 0.3623
Epoch 6/100
29/29 [==============================] - 1s 19ms/step - loss: 1.0833 - accuracy: 0.3928 - val_loss: 1.0843 - val_accuracy: 0.3948
Epoch 7/100
29/29 [==============================] - 1s 19ms/step - loss: 1.0792 - accuracy: 0.4059 - val_loss: 1.0787 - val_accuracy: 0.4121
Epoch 

29/29 [==============================] - 1s 19ms/step - loss: 0.2165 - accuracy: 0.9170 - val_loss: 0.3485 - val_accuracy: 0.8351
Epoch 59/100
29/29 [==============================] - 1s 20ms/step - loss: 0.2329 - accuracy: 0.9045 - val_loss: 0.3267 - val_accuracy: 0.8503
Epoch 60/100
29/29 [==============================] - 1s 20ms/step - loss: 0.2388 - accuracy: 0.9007 - val_loss: 0.4558 - val_accuracy: 0.8156
Epoch 61/100
29/29 [==============================] - 1s 20ms/step - loss: 0.2826 - accuracy: 0.8812 - val_loss: 0.3798 - val_accuracy: 0.8286
Epoch 62/100
29/29 [==============================] - 1s 20ms/step - loss: 0.2039 - accuracy: 0.9186 - val_loss: 0.3523 - val_accuracy: 0.8655
Epoch 63/100
29/29 [==============================] - 1s 20ms/step - loss: 0.2170 - accuracy: 0.9105 - val_loss: 0.3232 - val_accuracy: 0.8547
Epoch 64/100
29/29 [==============================] - 1s 20ms/step - loss: 0.2288 - accuracy: 0.9002 - val_loss: 0.3886 - val_accuracy: 0.8330
Epoch 65/100